# Empty List Filter

This is a technique using [this keras classification open kernel](https://www.kaggle.com/mobassir/keras-efficientnetb2-for-classifying-cloud).

We didn't retrain the classifier, just clear some over confident predictions

In [56]:
import pandas as pd
import numpy as np
from argparse import ArgumentParser

In [57]:
# ap = ArgumentParser()

# ap.add_argument("--csv", dest = "csv", type = str, help = "csv file path")
# args = ap.parse_args()

# SUBFILE = args.csv

SUBFILE = "convex_csvs_1117_012120_sub.csv"

submission = pd.read_csv(SUBFILE)

In [58]:
sub_ = pd.read_csv(SUBFILE)
sub_["img"] = sub_.Image_Label.apply(lambda x:x.split("_")[0])
sub_["lbl"] = sub_.Image_Label.apply(lambda x:x.split("_")[1])
sub_["has_class"] = sub_.EncodedPixels.isnull()*1

In [59]:
class_counter = sub_.groupby("img").sum().reset_index()

In [60]:
missing = class_counter[class_counter.has_class==0]

In [61]:
img2counter = dict(zip(class_counter["img"],class_counter["has_class"]))

In [62]:
sub_["counter"] = sub_.img.apply(lambda x: img2counter[x])

In [63]:
sub_

,Image_Label,EncodedPixels,img,lbl,has_class,counter
0,002f507.jpg_Fish,NaN,002f507.jpg,Fish,1,3
1,002f507.jpg_Flower,NaN,002f507.jpg,Flower,1,3
2,002f507.jpg_Gravel,1 338 351 342 701 343 1051 345 1401 346 1751 3...,002f507.jpg,Gravel,0,3
3,002f507.jpg_Sugar,NaN,002f507.jpg,Sugar,1,3
4,0035ae9.jpg_Fish,NaN,0035ae9.jpg,Fish,1,2
...,...,...,...,...,...,...
14787,ffe0620.jpg_Sugar,NaN,ffe0620.jpg,Sugar,1,2
14788,fff3c5c.jpg_Fish,505 46 847 85 1192 112 1539 129 1887 141 2236 ...,fff3c5c.jpg,Fish,0,2
14789,fff3c5c.jpg_Flower,NaN,fff3c5c.jpg,Flower,1,2
14790,fff3c5c.jpg_Gravel,NaN,fff3c5c.jpg,Gravel,1,2


In [64]:
image_labels_empty = set(pd.read_csv("empty_list.csv")["Empty"])
print("Picture x class that suppose to be empty:\t%s"%(len(image_labels_empty)))

Picture x class that suppose to be empty:	4067


Non Empety For Now

In [65]:
predictions_nonempty = set(sub_.loc[~sub_['EncodedPixels'].isnull(), 'Image_Label'].values)
nonempty_single = set(sub_.loc[((~sub_['EncodedPixels'].isnull())& sub_.counter==1), 'Image_Label'].values)
print("Mask we have:\t%s"%(len(predictions_nonempty)))
print("Non empty single %s"%(len(nonempty_single)))

Mask we have:	6445
Non empty single 2565


In [66]:
nonempty_can_remove = predictions_nonempty-nonempty_single

In [67]:
remove_list = image_labels_empty.intersection(nonempty_can_remove)

In [68]:
print(f'{len(image_labels_empty.intersection(predictions_nonempty))} masks are in intersection')
print(f'{len(image_labels_empty.intersection(nonempty_single))} masks are in intersection & single')
print(f'{len(image_labels_empty.intersection(nonempty_can_remove))} masks will be removed')

96 masks are in intersection
52 masks are in intersection & single
44 masks will be removed


In [70]:
#removing masks
submission.loc[submission['Image_Label'].isin(remove_list), 'EncodedPixels'] = np.nan
SAVE_PATH = 'emp_%s'%(SUBFILE)
print("New submission saved to :\t%s"%(SAVE_PATH))
submission.to_csv(SAVE_PATH, index=None)

New submission saved to :	emp_convex_csvs_1117_012120_sub.csv
